In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
import seaborn as sns

# 新しく実装したコンポーネント
from nm.research.global_ppm.utils.company_code_manager import (
    CompanyCodeManager, get_company_code_manager
)
from nm.research.global_ppm.data_provider.msci_data_provider import MSCIDataProvider
from nm.research.global_ppm.py.beta_calculator import (
    BetaCalculator, BetaType, ReturnFrequency
)

# 既存のコンポーネント
from nm.research.global_ppm.py.wacc_calculator import WACCCalculator
from nm.research.global_ppm.utils.country_risk_parameters import CountryRiskParams

# 表示設定
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline


In [ ]:
# CompanyCodeManagerの初期化
try:
    company_manager = get_company_code_manager()
    
    # キャッシュ統計の表示
    stats = company_manager.get_cache_statistics()
    print("=== CompanyCodeManager キャッシュ統計 ===")
    for key, value in stats.items():
        print(f"{key}: {value}")
    
    # サンプル企業の表示（デバッグ用）
    print("\n=== サンプル企業データ ===")
    samples = company_manager.get_sample_companies(3)
    for i, company in enumerate(samples, 1):
        print(f"\n企業 {i}:")
        print(f"  FSYM_ID: {company.fsym_id}")
        print(f"  Entity ID: {company.factset_entity_id}")
        print(f"  会社名: {company.company_name}")
        print(f"  ISIN: {company.isin}")
        print(f"  SEDOL: {company.sedol}")
        print(f"  CUSIP: {company.cusip}")
        print(f"  Ticker: {company.ticker}")
        print(f"  本社所在国: {company.headquarters_country_code}")
    
    # 既存の使用例（データがある場合のみ実行）
    if samples:
        print("\n=== コード変換テスト ===")
        test_company = samples[0]
        if test_company.isin and test_company.fsym_id:
            print(f"ISIN {test_company.isin} → FSYM_ID: {test_company.fsym_id}")
            
            # 逆変換のテスト
            converted_isin = company_manager.convert_code(test_company.fsym_id, 'fsym', 'isin')
            print(f"FSYM_ID {test_company.fsym_id} → ISIN: {converted_isin}")
    
    # 企業検索
    print("\n=== 企業検索テスト ===")
    search_results = company_manager.search_companies("Nintendo")
    print(f"'Nintendo'の検索結果: {len(search_results)} 件")
    for company in search_results[:3]:  # 最初の3件を表示
        print(f"- {company.company_name} (ISIN: {company.isin})")

except Exception as e:
    print(f"エラーが発生しました: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
# MSCIDataProviderの初期化
msci_provider = MSCIDataProvider()

# 使用例1: MSCI World Indexの構成銘柄を取得
world_constituents = msci_provider.get_index_constituents(
    index_name='WORLD',
    as_of_date=date(2024, 1, 31)  # 2024年1月末時点
)

print(f"MSCI World Index 構成銘柄数: {len(world_constituents)}")
print(f"\n上位10銘柄（時価総額順）:")
print(world_constituents[['COMPANY_NAME', 'ISO_COUNTRY_SYMBOL', 'MARKET_CAP_USD', 'SECTOR']].head(10))

# 使用例2: 国別の構成銘柄数を集計
country_summary = world_constituents.groupby('ISO_COUNTRY_SYMBOL').agg({
    'MSCI_SECURITY_CODE': 'count',
    'MARKET_CAP_USD': 'sum'
}).rename(columns={'MSCI_SECURITY_CODE': 'COUNT'})
country_summary = country_summary.sort_values('MARKET_CAP_USD', ascending=False)

print(f"\n国別構成銘柄サマリー（上位10か国）:")
print(country_summary.head(10))


In [ ]:
# 使用例3: インデックスリターンの取得
index_returns = msci_provider.get_index_returns(
    index_name='WORLD',
    start_date=date(2023, 1, 1),
    end_date=date(2024, 1, 31),
    frequency='monthly'
)

# リターンの可視化
if not index_returns.empty:
    plt.figure(figsize=(12, 6))
    plt.plot(index_returns['DATA_DATE'], index_returns['USD_RETURN'], marker='o')
    plt.title('MSCI World Index 月次リターン (USD)')
    plt.xlabel('日付')
    plt.ylabel('リターン (%)')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # 統計サマリー
    print(f"平均月次リターン: {index_returns['USD_RETURN'].mean():.2f}%")
    print(f"月次ボラティリティ: {index_returns['USD_RETURN'].std():.2f}%")
    print(f"年率換算ボラティリティ: {index_returns['USD_RETURN'].std() * np.sqrt(12):.2f}%")


In [ ]:
# BetaCalculatorの初期化
beta_calculator = BetaCalculator()

# 使用例1: 個別証券のベータ計算
# Apple Inc.のISINを使用
apple_isin = "US0378331005"

beta_result = beta_calculator.calculate_beta(
    security_id=apple_isin,
    identifier_type='isin',
    index_name='WORLD',
    beta_type=BetaType.GLOBAL,
    frequency=ReturnFrequency.MONTHLY,
    lookback_years=5
)

if beta_result:
    print(f"企業: {beta_result.security_id}")
    print(f"ベータ値: {beta_result.beta:.3f}")
    print(f"R二乗: {beta_result.r_squared:.3f}")
    print(f"相関係数: {beta_result.correlation:.3f}")
    print(f"観測数: {beta_result.num_observations}")
    print(f"計算期間: {beta_result.start_date} ～ {beta_result.end_date}")
else:
    print("ベータ計算に失敗しました")


In [ ]:
# 使用例2: 複数企業のベータ一括計算
sample_securities = [
    "US0378331005",  # Apple
    "US5949181045",  # Microsoft
    "US02079K3059",  # Alphabet
    "JP3633400001",  # Toyota
    "GB0031348658"   # Barclays
]

batch_results = beta_calculator.calculate_batch_beta(
    securities=sample_securities,
    identifier_type='isin',
    index_name='WORLD',
    frequency=ReturnFrequency.MONTHLY,
    lookback_years=3
)

# 結果の表示
print("ベータ計算結果:")
if not batch_results.empty:
    display_cols = ['security_id', 'beta', 'r_squared', 'num_observations', 'security_volatility']
    print(batch_results[display_cols].round(3))
    
    # ベータ値の分布を可視化
    plt.figure(figsize=(10, 6))
    valid_betas = batch_results.dropna(subset=['beta'])
    if not valid_betas.empty:
        plt.bar(range(len(valid_betas)), valid_betas['beta'])
        plt.axhline(y=1.0, color='r', linestyle='--', label='市場ベータ (1.0)')
        plt.xticks(range(len(valid_betas)), valid_betas['security_id'], rotation=45)
        plt.ylabel('ベータ値')
        plt.title('企業別ベータ値')
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
# 財務データの取得（仮のデータを作成）
# 実際の使用では、FinancialDataProviderから取得
sample_financial_data = pd.DataFrame({
    'FACTSET_ENTITY_ID': ['ENTITY_001', 'ENTITY_002', 'ENTITY_003'],
    'ISO_COUNTRY': ['US', 'JP', 'GB'],
    'FTERM_2': [202312, 202312, 202312],
    'FF_MKT_VAL': [1000000, 500000, 300000],  # 時価総額（百万ドル）
    'FF_DEBT': [400000, 200000, 150000],      # 総負債（百万ドル）
    'FF_TAX_RATE': [0.21, 0.30, 0.25],        # 法人税率
    'FF_EFF_INT_RATE': [0.04, 0.02, 0.05],    # 実効金利
    'FF_INT_EXP_DEBT': [16000, 4000, 7500]    # 支払利息
})

# ベータ値を追加（実際の計算結果を使用）
beta_values = pd.Series([1.2, 0.8, 1.1], index=sample_financial_data.index)

# WACCCalculatorの初期化と国別パラメータ設定
wacc_calculator = WACCCalculator()

# 国別リスクパラメータの設定
country_params = {
    'US': CountryRiskParams(
        risk_free_rate=0.045,      # 4.5%
        market_risk_premium=0.065,  # 6.5%
        country_tax_rate=0.21       # 21%
    ),
    'JP': CountryRiskParams(
        risk_free_rate=0.005,      # 0.5%
        market_risk_premium=0.060,  # 6.0%
        country_tax_rate=0.30       # 30%
    ),
    'GB': CountryRiskParams(
        risk_free_rate=0.040,      # 4.0%
        market_risk_premium=0.065,  # 6.5%
        country_tax_rate=0.25       # 25%
    )
}

wacc_calculator.set_country_params(country_params)

# WACC計算の実行
result_data = sample_financial_data.copy()
result_data = wacc_calculator.calculate_market_value_equity(result_data)
result_data = wacc_calculator.calculate_average_debt(result_data)
result_data = wacc_calculator.calculate_cost_of_debt(result_data)

# ベータを使用したWACC計算
wacc_results = wacc_calculator.calculate_wacc(
    result_data,
    beta=beta_values,
    use_country_params=True
)

# 結果の表示
print("WACC計算結果:")
display_columns = [
    'FACTSET_ENTITY_ID', 'ISO_COUNTRY', 
    '株主資本コスト', '負債コスト（税引後）', 'WACC',
    '株主資本比率', '負債比率'
]
print(wacc_results[display_columns].round(3))
